## Müllabfuhr


[Aufgabe](muellabfuhr.pdf) - [Lösungshinweise](muellabfuhrL.pdf)

Beispiel 1:

<img src='bild2.png' width='400'>

### Kürzeste Wege mit Floyd

Der Floyd-Algorithmus liefert uns kürzeste Wege zwischen zwei beliebigen Knoten.
Die beiden beiden berechneten Matrizen D (Distanzmatrix) und P (Vorgängerknoten des kürzesten Wegs)
speichern wir als globale Variablen. Der Floyd-Algorithmus erwartet als Eingabe eine Adjazenzmatrix


#### Adjazenzmatrix erstellen

In [209]:
inf = float('inf')
nr = 6
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

GM = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    GM[von][bis]=km
    GM[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    GM[i][i]=0

In [ ]:
GM

In [210]:
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(GM)

In [ ]:
printMatrix(D)

Jetzt können wir die Distanz und die kürzesten Wege zwischen zwei beliebigen Knoten berechnen.

In [211]:
# die Matrizen D und P werden als globale Variablen vorausgesetzt.
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2  
    
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

In [9]:
print(sPath(0,2))
print(dist(0,2))

(0, 93, 45, 42, 5, 2)
54730


### Kreis, der alle Kanten durchläuft.

Ein Kreis, der alle Kanten nur einmal durchläuft, ist ein Eulerkreis. Der existiert genau dann, wenn wir alle Knoten einen geraden Grad haben. Das können wir bei unseren Graphen nicht erwarten. Wir verbinden die Knoten, die ungeraden Grad haben paarweise miteinander (möglichst billige Strecken), dann konstruieren wir den Eulerkreis. Dazu benötigen wir eine Modellierung des Graphen als Adjazenzliste. Allerdings brauchen wir hier die Gewichte nicht berücksichtigen (wir wollen ja alle Kanten durchlaufen, egal was sie für ein Gewicht haben). Nur beim matching der ungeraden Kanten benötigen wir die Information über die kürzesten Wege. Die holen wir uns aus dem Ergebnis des Floyd-Algorithmus. 

#### Adjazenzliste erstellen

In [212]:
nr = 6
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:set() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, _ = f.readline().split()
    v1, v2 = int(v1), int(v2)
    G[v1].add(v2)
    G[v2].add(v1)

In [213]:
G

{0: {4, 44, 58, 71, 93},
 1: {3, 11, 49},
 2: {5, 13, 19, 72, 84},
 3: {1, 11, 14, 49, 68},
 4: {0, 10, 44, 93, 98},
 5: {2, 13, 41, 42, 67, 77, 86},
 6: {26, 62},
 7: {12, 15, 50},
 8: {20, 34, 87},
 9: {17, 22, 26, 47, 54, 56},
 10: {4, 45, 48, 98},
 11: {1, 3, 36, 37},
 12: {7, 15, 21, 50, 85},
 13: {2, 5, 72, 77, 84},
 14: {3, 36, 68},
 15: {7, 12, 50, 85, 94},
 16: {74, 80, 89, 92},
 17: {9, 22, 54, 56},
 18: {23, 77},
 19: {2, 39, 48, 84},
 20: {8, 34, 52, 73},
 21: {12, 49, 50},
 22: {9, 17, 24, 47, 54},
 23: {18, 38, 69, 75, 78, 97},
 24: {22, 31, 59, 61, 82},
 25: {30, 39, 48, 83},
 26: {6, 9, 29, 47, 56, 57},
 27: {43, 87, 94, 99},
 28: {30, 31, 70, 82, 83},
 29: {26, 53, 57},
 30: {25, 28, 33, 39, 83},
 31: {24, 28, 70, 82},
 32: {35, 46, 81, 88, 90, 98},
 33: {30, 60, 83},
 34: {8, 20, 36},
 35: {32, 46, 88, 93, 98},
 36: {11, 14, 34, 55},
 37: {11, 68, 78},
 38: {23, 69, 75, 97},
 39: {19, 25, 30, 60, 83},
 40: {66, 89, 99},
 41: {5, 42, 45, 67, 86, 90},
 42: {5, 41, 45, 6

#### Knoten mit ungeradem Grad bestimmen

In [215]:
U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
print(U)
print(len(U))

[0, 1, 2, 3, 4, 5, 7, 8, 12, 13, 14, 15, 21, 22, 24, 28, 29, 30, 33, 34, 35, 37, 39, 40, 42, 43, 46, 48, 49, 51, 52, 55, 57, 60, 62, 63, 64, 68, 69, 70, 72, 73, 75, 78, 81, 83, 86, 87, 88, 90, 91, 95, 98, 99]
54


#### Greedy Matching für die Knoten mit ungeradem Grad

```
Setze result = []
Solange U noch nicht leer:
    hole ein Element v1 aus U
    wähle aus dasjenige Element v2, das zu v1 den geringsten Abstand hat
    Füge (v1,v2) in die Liste result ein.
    Lösche v1, v2 aus U
```

In [216]:
def match(U):
    U1 = U.copy()          # U soll unverändert bleiben
    result = []
    while U1:
        v = U1.pop()       # v1 ist damit automatisch aus U gelöscht
        best_val = inf
        best = None
        for w in U1:
            if dist(v,w) < best_val:
                best_val = dist(v,w)
                best = w
        result.append((v,best))
        U1.remove(best)
    return result     

In [217]:
match(U) 

[(99, 40),
 (98, 35),
 (95, 29),
 (91, 68),
 (90, 88),
 (87, 43),
 (86, 5),
 (83, 30),
 (81, 51),
 (78, 75),
 (73, 52),
 (72, 13),
 (70, 28),
 (69, 55),
 (64, 63),
 (62, 22),
 (60, 39),
 (57, 24),
 (49, 1),
 (48, 42),
 (46, 0),
 (37, 3),
 (34, 8),
 (33, 4),
 (21, 12),
 (15, 7),
 (14, 2)]

Kosten des Matchings:


In [218]:
def matchCost(U):
    cost = 0
    for v,w in match(U):
        print(dist(v,w))
        cost += dist(v,w)
    return cost

print(f'Kosten des Matchings {matchCost(U)}')

3284
1367
10479
8294
6562
11171
8562
5381
14840
13369
12591
9524
5406
27962
18034
15566
10998
51222
4180
25988
14963
17522
3519
24477
12039
5807
99165
Kosten des Matchings 442272


#### Einfügen der Metakanten in den Graphen
Es kann sein, dass eine Kante eingefügt werden soll zwischen zwei Knoten, die schon eine Kante haben.
Unsere Implementation des Graphen unterstützt keine doppelten Kanten. Deswegen fügen wir auch einen Metaknoten zwischendrin ein. Wir nummerieren in ab 10000 aufwärts um ihn als Metaknoten erkennen zu können.

In [219]:
def addMeta(U):
    global hilf
    for e in match(U):
        v1, v2 = e
        G[hilf] = {v1,v2}
        G[v1].add(hilf)
        G[v2].add(hilf)
        hilf+=1


In [220]:
hilf = 10000
addMeta(U)
G

{0: {4, 44, 58, 71, 93, 10020},
 1: {3, 11, 49, 10018},
 2: {5, 13, 19, 72, 84, 10026},
 3: {1, 11, 14, 49, 68, 10021},
 4: {0, 10, 44, 93, 98, 10023},
 5: {2, 13, 41, 42, 67, 77, 86, 10006},
 6: {26, 62},
 7: {12, 15, 50, 10025},
 8: {20, 34, 87, 10022},
 9: {17, 22, 26, 47, 54, 56},
 10: {4, 45, 48, 98},
 11: {1, 3, 36, 37},
 12: {7, 15, 21, 50, 85, 10024},
 13: {2, 5, 72, 77, 84, 10011},
 14: {3, 36, 68, 10026},
 15: {7, 12, 50, 85, 94, 10025},
 16: {74, 80, 89, 92},
 17: {9, 22, 54, 56},
 18: {23, 77},
 19: {2, 39, 48, 84},
 20: {8, 34, 52, 73},
 21: {12, 49, 50, 10024},
 22: {9, 17, 24, 47, 54, 10015},
 23: {18, 38, 69, 75, 78, 97},
 24: {22, 31, 59, 61, 82, 10017},
 25: {30, 39, 48, 83},
 26: {6, 9, 29, 47, 56, 57},
 27: {43, 87, 94, 99},
 28: {30, 31, 70, 82, 83, 10012},
 29: {26, 53, 57, 10002},
 30: {25, 28, 33, 39, 83, 10007},
 31: {24, 28, 70, 82},
 32: {35, 46, 81, 88, 90, 98},
 33: {30, 60, 83, 10023},
 34: {8, 20, 36, 10022},
 35: {32, 46, 88, 93, 98, 10001},
 36: {11, 14

#### Eulerkreis erstellen
Jetzt haben alle Knoten einen geraden Grad und wir können den Eulerkreis erstellen.

In [221]:
import sys
from copy import deepcopy

GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 

def dfs(v):

    while degrees[v] != 0:
        next = list(GE[v])[0]    # die erste noch vorhandene Kante
        GE[v].remove(next)       # wird aus dem Graphen gelöscht
        GE[next].remove(v)       # von beiden Seiten
        degrees[v]-=1            # der Grad der Knoten v und next
        degrees[next]-=1         # wird erniedrigt
        dfs(next)
    path.append(v) 
    
path = []
dfs(0)                         # beim undirected eulercircle unnötig
print(path)    

[0, 58, 93, 45, 58, 98, 93, 4, 44, 60, 61, 24, 31, 70, 10012, 28, 31, 82, 28, 30, 25, 39, 60, 59, 62, 54, 22, 17, 54, 9, 26, 29, 53, 95, 10002, 29, 57, 26, 47, 22, 10015, 62, 6, 26, 56, 17, 9, 56, 47, 9, 22, 24, 59, 44, 0, 71, 46, 10020, 0, 93, 35, 88, 90, 10004, 88, 81, 10008, 51, 81, 76, 63, 10014, 64, 63, 80, 53, 57, 92, 74, 16, 92, 79, 57, 10017, 24, 82, 70, 28, 83, 30, 10007, 83, 39, 30, 33, 10023, 4, 10, 45, 42, 86, 77, 84, 19, 2, 84, 13, 77, 18, 23, 78, 10009, 75, 55, 78, 37, 10021, 3, 49, 91, 68, 10003, 91, 50, 85, 87, 27, 94, 15, 85, 12, 21, 50, 15, 12, 50, 7, 15, 10025, 7, 12, 10024, 21, 49, 68, 14, 36, 55, 10013, 69, 78, 75, 23, 38, 69, 75, 38, 97, 69, 23, 97, 55, 52, 10010, 73, 52, 20, 34, 10022, 8, 87, 10005, 43, 99, 27, 43, 66, 40, 99, 89, 96, 99, 10000, 40, 89, 16, 80, 76, 65, 51, 64, 80, 65, 81, 32, 90, 41, 86, 10006, 5, 86, 72, 13, 5, 77, 67, 5, 42, 41, 45, 98, 10001, 35, 46, 32, 88, 73, 20, 8, 34, 36, 11, 37, 68, 3, 11, 1, 10018, 49, 1, 3, 14, 10026, 2, 13, 10011, 72,

In [222]:
len(path)

259

Links und rechts der 10000er Zahlen stehen die Knoten, zwischen die wir den kürzesten Weg zueinander schreiben müssen.

In [223]:
def eliminateMeta(path):
    result = []
    for i in range(1,len(path)-1):
        if path[i] < 10000:
            result.append(path[i])
        else:
            v1, v2 = path[i-1],path[i+1]
            for v in sPath(v1,v2)[1:-1]:
                result.append(v)
    return result

In [225]:
path = eliminateMeta(path)
print(path)
print(len(path))

[58, 93, 45, 58, 98, 93, 4, 44, 60, 61, 24, 31, 70, 28, 31, 82, 28, 30, 25, 39, 60, 59, 62, 54, 22, 17, 54, 9, 26, 29, 53, 95, 53, 29, 57, 26, 47, 22, 54, 62, 6, 26, 56, 17, 9, 56, 47, 9, 22, 24, 59, 44, 0, 71, 46, 71, 0, 93, 35, 88, 90, 88, 81, 51, 81, 76, 63, 64, 63, 80, 53, 57, 92, 74, 16, 92, 79, 57, 26, 47, 22, 24, 82, 70, 28, 83, 30, 83, 39, 30, 33, 60, 44, 4, 10, 45, 42, 86, 77, 84, 19, 2, 84, 13, 77, 18, 23, 78, 75, 55, 78, 37, 68, 3, 49, 91, 68, 91, 50, 85, 87, 27, 94, 15, 85, 12, 21, 50, 15, 12, 50, 7, 15, 7, 12, 21, 49, 68, 14, 36, 55, 97, 69, 78, 75, 23, 38, 69, 75, 38, 97, 69, 23, 97, 55, 52, 73, 52, 20, 34, 8, 87, 27, 43, 99, 27, 43, 66, 40, 99, 89, 96, 99, 40, 89, 16, 80, 76, 65, 51, 64, 80, 65, 81, 32, 90, 41, 86, 5, 86, 72, 13, 5, 77, 67, 5, 42, 41, 45, 98, 35, 46, 32, 88, 73, 20, 8, 34, 36, 11, 37, 68, 3, 11, 1, 49, 1, 3, 14, 36, 34, 20, 73, 88, 90, 41, 5, 2, 13, 72, 2, 5, 41, 67, 42, 45, 10, 48, 19, 39, 60, 33, 83, 25, 48, 10, 98, 35, 32, 98, 4]
251


#### Den Eulerkreis aufteilen

In [226]:
def abschnitt(p,i,j):
    '''
    p: Pfad als Tuple oder Liste
    i, j: Indizes des Pfads, i <= j
    returns: Pfad (Liste) bestehend aus kürzestem Weg von 0 nach p[i], dann weiter auf p bis p[j], dann den
      kürzesten Weg p[j] zurück nach 0 
    '''
    return sPath(0,p[i])+tuple(p[i+1:j])+sPath(p[j],0)

def pathCost(p):
    '''
    p: Pfad
    returns: Summe der Kantenkosten entlang des Pfades
    '''
    pcost = 0                                  
    for k in range(len(p)-1):
        pcost += GM[p[k]][p[k+1]] 
    return pcost

def aufteilung(p,n):
    '''
    Aufteilung von Pfad p in 5 Abschnitte, wobei jeder Abschnitt <= n kostet
    returns: Liste mit Abschnitten der Indizes von Pfad p. Wenn n zu klein ist,
    decken die Abschnitte nicht den ganzen Pfad ab.
    z.B: [(0, 6), (6, 7), (7, 9), (9, 12), (12, 14)]
    '''
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(p) and (pathCost(abschnitt(p,i,j))) <= n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

def aufteilungOK(p,n):
    '''
    p: Pfad
    returns True, wenn die Aufteilung mit n den ganzen Pfad überdeckt
    '''
    au = aufteilung(path,n)
    lastIndex = au[-1][1]
    return lastIndex == len(path)-1

def printPath(p):
    '''
    printed den Pfad p mit seinen Kosten
    '''
    cost = 0
    for i in range(len(p)-1):
        
        cost += D[p[i]][p[i+1]]
    print(cost)

In [114]:
p = [0, 5, 7, 6, 3, 6, 7, 4, 5, 3, 4, 0, 6, 1, 3, 2, 3, 6, 0]
abschnitt(p,6,10)

(0, 58, 98, 35, 88, 73, 20, 8, 87, 85, 12, 7, 4, 5, 3, 4, 0)

In [86]:
sPath(3,0)

(3, 14, 36, 34, 20, 73, 88, 35, 98, 58, 0)

In [87]:
print(aufteilung(path,18), aufteilungOK(path,18))
print(aufteilung(path,25), aufteilungOK(path,25))
 

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)] False
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)] False


#### Binäre Suche, um die beste Aufteilung des Eulerkreises zu finden

In [227]:
def besteAufteilung(path):

    mincost = 0
    maxcost = pathCost(abschnitt(path,0,len(path)-1))

    best = maxcost
    versuch = maxcost//2

    while True:

        if aufteilungOK(path,versuch):
            best = versuch
            versuch = versuch//2
        else:
            versuch1 = (versuch+best) // 2
            if versuch1 == versuch: 
                 break
            versuch = versuch1
            
    return best, aufteilung(path,best)

 

In [228]:
best, teile =   besteAufteilung(path)
print('Länge des Pfads',len(path))
print('Beste Aufteilung',teile)
print('Maximale Pfadkosten bei bester Aufteilung:', best)

Länge des Pfads 251
Beste Aufteilung [(0, 62), (62, 104), (104, 148), (148, 196), (196, 250)]
Maximale Pfadkosten bei bester Aufteilung: 594484


#### Fahrplan für die 5 Müllabfuhren

In [229]:
print(f'Fahrplan für Beispiel {nr}:')
maxcost = 0
for teil in teile:
    p = abschnitt(path,teil[0],teil[1])
    cost = pathCost(p)
    if cost > maxcost:
        maxcost = cost
    print(f'Kosten: {cost} - {p}')
print(f'Maximale Kosten einer Fahrt: {maxcost}')

Fahrplan für Beispiel 6:
Kosten: 570333 - (0, 58, 93, 45, 58, 98, 93, 4, 44, 60, 61, 24, 31, 70, 28, 31, 82, 28, 30, 25, 39, 60, 59, 62, 54, 22, 17, 54, 9, 26, 29, 53, 95, 53, 29, 57, 26, 47, 22, 54, 62, 6, 26, 56, 17, 9, 56, 47, 9, 22, 24, 59, 44, 0, 71, 46, 71, 0, 93, 35, 88, 90, 88, 81, 32, 98, 58, 0)
Kosten: 594484 - (0, 58, 98, 32, 81, 51, 81, 76, 63, 64, 63, 80, 53, 57, 92, 74, 16, 92, 79, 57, 26, 47, 22, 24, 82, 70, 28, 83, 30, 83, 39, 30, 33, 60, 44, 4, 10, 45, 42, 86, 77, 84, 19, 2, 84, 13, 77, 5, 42, 45, 93, 0)
Kosten: 583752 - (0, 93, 45, 42, 5, 77, 18, 23, 78, 75, 55, 78, 37, 68, 3, 49, 91, 68, 91, 50, 85, 87, 27, 94, 15, 85, 12, 21, 50, 15, 12, 50, 7, 15, 7, 12, 21, 49, 68, 14, 36, 55, 97, 69, 78, 75, 23, 38, 69, 75, 55, 52, 73, 88, 35, 98, 58, 0)
Kosten: 592900 - (0, 58, 98, 35, 88, 73, 52, 55, 75, 38, 97, 69, 23, 97, 55, 52, 73, 52, 20, 34, 8, 87, 27, 43, 99, 27, 43, 66, 40, 99, 89, 96, 99, 40, 89, 16, 80, 76, 65, 51, 64, 80, 65, 81, 32, 90, 41, 86, 5, 86, 72, 13, 5, 77,

### Das vollständige Programm

In [208]:
from copy import deepcopy
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2  
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

def match(U):
    U1 = U.copy()          # U soll unverändert bleiben
    result = []
    while U1:
        v = U1.pop()       # v1 ist damit automatisch aus U gelöscht
        best_val = inf
        best = None
        for w in U1:
            if dist(v,w) < best_val:
                best_val = dist(v,w)
                best = w
        result.append((v,best))
        U1.remove(best)
    return result  

def addMeta(U):
    global hilf
    for e in match(U):
        v1, v2 = e
        G[hilf] = {v1,v2}
        G[v1].add(hilf)
        G[v2].add(hilf)
        hilf+=1

def dfs(v):
    while degrees[v] != 0:
        next = list(GE[v])[0]    # die erste noch vorhandene Kante
        GE[v].remove(next)       # wird aus dem Graphen gelöscht
        GE[next].remove(v)       # von beiden Seiten
        degrees[v]-=1            # der Grad der Knoten v und next
        degrees[next]-=1         # wird erniedrigt
        dfs(next)
    path.append(v) 

def eliminateMeta(path):
    result = []
    for i in range(1,len(path)-1):
        if path[i] < 10000:
            result.append(path[i])
        else:
            v1, v2 = path[i-1],path[i+1]
            for v in sPath(v1,v2)[1:-1]:
                result.append(v)
    return result

def abschnitt(p,i,j):
    '''
    p: Pfad als Tuple oder Liste
    i, j: Indizes des Pfads, i <= j
    returns: Pfad (Liste) bestehend aus kürzestem Weg von 0 nach p[i], dann weiter auf p bis p[j], dann den
      kürzesten Weg p[j] zurück nach 0 
    '''
    return sPath(0,p[i])+tuple(p[i+1:j])+sPath(p[j],0)

def pathCost(p):
    '''
    p: Pfad
    returns: Summe der Kantenkosten entlang des Pfades
    '''
    pcost = 0                                  
    for k in range(len(p)-1):
        pcost += GM[p[k]][p[k+1]] 
    return pcost

def aufteilung(p,n):
    '''
    Aufteilung von Pfad p in 5 Abschnitte, wobei jeder Abschnitt <= n kostet
    returns: Liste mit Abschnitten der Indizes von Pfad p. Wenn n zu klein ist,
    decken die Abschnitte nicht den ganzen Pfad ab.
    z.B: [(0, 6), (6, 7), (7, 9), (9, 12), (12, 14)]
    '''
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(p) and (pathCost(abschnitt(p,i,j))) <= n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

def aufteilungOK(p,n):
    '''
    p: Pfad
    returns True, wenn die Aufteilung mit n den ganzen Pfad überdeckt
    '''
    au = aufteilung(path,n)
    lastIndex = au[-1][1]
    return lastIndex == len(path)-1

def printPath(p):
    '''
    printed den Pfad p mit seinen Kosten
    '''
    cost = 0
    for i in range(len(p)-1):
        
        cost += D[p[i]][p[i+1]]
    print(cost)
    
def besteAufteilung(path):

    mincost = 0
    maxcost = pathCost(abschnitt(path,0,len(path)-1))

    best = maxcost
    versuch = maxcost//2

    while True:

        if aufteilungOK(path,versuch):
            best = versuch
            versuch = versuch//2
        else:
            versuch1 = (versuch+best) // 2
            if versuch1 == versuch: 
                 break
            versuch = versuch1
            
    return best, aufteilung(path,best)


nr = 1
inf = float('inf')
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

GM = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    GM[von][bis]=km
    GM[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    GM[i][i]=0
    
D, P = floyd(GM)

f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:set() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, _ = f.readline().split()
    v1, v2 = int(v1), int(v2)
    G[v1].add(v2)
    G[v2].add(v1)
    
U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
        
hilf = 10000
addMeta(U)
GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 
path = []
dfs(0)                         # beim undirected eulercircle unnötig
path = eliminateMeta(path)
kosten, teile =   besteAufteilung(path)

print(f'Fahrplan für Beispiel {nr}:')
maxcost = 0
for teil in teile:
    p = abschnitt(path,teil[0],teil[1])
    cost = pathCost(p)
    if cost > maxcost:
        maxcost = cost
    print(f'Kosten: {cost} - {p}')
print(f'Maximale Kosten einer Fahrt: {maxcost}')

Fahrplan für Beispiel 1:
Kosten: 18 - (0, 6, 7, 5, 4, 3, 6, 0)
Kosten: 18 - (0, 6, 3, 2, 3, 6, 0)
Kosten: 18 - (0, 6, 1, 3, 5, 7, 6, 0)
Kosten: 10 - (0, 6, 7, 5, 0)
Kosten: 16 - (0, 4, 7, 6, 0)
Maximale Kosten einer Fahrt: 18


#### Ergebnisse
```
Fahrplan für Beispiel 1:
Kosten: 18 - (0, 6, 7, 5, 4, 3, 6, 0)
Kosten: 18 - (0, 6, 3, 2, 3, 6, 0)
Kosten: 18 - (0, 6, 1, 3, 5, 7, 6, 0)
Kosten: 10 - (0, 6, 7, 5, 0)
Kosten: 16 - (0, 4, 7, 6, 0)
Maximale Kosten einer Fahrt: 18

```